# PART 1 – DATA

a- Create an SQL query that provides one of our main business KPIs - the number of Monthly
Active Users (MAU) per day per offer and per country
    
    A user is defined as Active if he streamed at least one track longer than 30 seconds
    
    MAU are based on a rolling 30 days period

b- Create an SQL query that provides the list of the 10 best streamers of Metronomy in 2016 in
France

c- Create an SQL query that provides the number of days between the inscription and the 100th
stream of all the Brazilian users registered in January 2018

# Import Libraries

In [1]:
import sqlite3
import pandas as pd

# Connection to the database

In [2]:
con = sqlite3.connect('music.db')

# Monthly Active Users (MAU) per day per offer and per country

Self join on the STREAMS table, with the condition that Stream_date1 - 30 < Stream_date2 <= Stream_date1
Please note that I have been using Sqlite, and for days difference the julianDay function was used.
However if it eas the like of Postgres, MySQL or others the syntaxt could be differnt (i.e. DATEDIFF, EXTRACT(MONTH FROM DATE),...)

In [3]:
MAU = pd.read_sql_query("SELECT ST1.Stream_date, ST1.Offer_id, ST1.Country, SUM(ST1.count) as MAU FROM \
                       (SELECT ST.Stream_date, Offer_id, U.Country, count(distinct ST.User_id) as count \
                        FROM STREAMS ST \
                        INNER JOIN USERS U ON U.User_id = ST.User_id \
                       WHERE Streams_duration >=30 \
                        GROUP BY Stream_date, Offer_id, U.Country ORDER BY Stream_date, Offer_id) ST1 \
                       JOIN \
                       (SELECT Stream_date, Offer_id, U.Country ,count(distinct ST.User_id) as count \
                        FROM STREAMS ST \
                        INNER JOIN USERS U ON U.User_id = ST.User_id \
                       WHERE Streams_duration >=30 \
                        GROUP BY Stream_date, Offer_id, U.Country ORDER BY Stream_date, Offer_id) ST2 \
                        ON \
                       julianDay(ST1.Stream_date) - julianDay(ST2.Stream_date) < 30 \
                        AND julianDay(ST1.Stream_date) - julianDay(ST2.Stream_date) >= 0 \
                       AND ST1.Offer_id = ST2.Offer_id \
                       AND ST1.Country = ST2.Country \
                       GROUP BY ST1.Stream_date, ST1.Offer_id, ST1.Country \
                       ", con)
MAU

,Stream_date,Offer_id,Country,MAU
0,2016-01-01,0,BR,1
1,2016-01-01,1,DE,1
2,2016-01-01,1,FR,1
3,2016-01-01,2,DE,1
4,2016-01-01,2,FR,2
5,2016-01-02,1,FR,2
6,2016-01-02,2,BR,3
7,2016-01-02,2,FR,2
8,2016-01-02,3,BR,1
9,2016-01-02,3,FR,1


# 10 best streamers of Metronomy in 2016 in France (streamed on French territory)

## Top streamers as time listened

In [4]:
metronomy_2016_FR_top_10_time = pd.read_sql_query("\
                        SELECT U.User_id, U.Email, U.Country, U.Gender, \
                        SUM(Streams_duration) as Total_listen FROM STREAMS as ST \
                        INNER JOIN SONGS as SG \
                        ON SG.Sng_id = ST.Sng_id \
                        INNER JOIN ARTISTS as A \
                        ON A.Artist_id = SG.Artist_id \
                        INNER JOIN USERS as U \
                        ON U.User_id = ST.User_id \
                        WHERE A.Artist_name = 'Metronomy' \
                        AND ST.Country = 'FR' \
                        AND strftime('%Y', ST.Stream_date)= '2016' \
                        GROUP by U.User_id, U.Email, U.Country, U.Gender \
                        ORDER BY Total_listen DESC LIMIT 10", con)

In [5]:
metronomy_2016_FR_top_10_time

,User_id,Email,Country,Gender,Total_listen
0,71,email71@gmail.com,BR,F,1482
1,77,email77@gmail.com,GB,F,1399
2,92,email92@gmail.com,BR,M,1271
3,29,email29@gmail.com,DE,M,1237
4,72,email72@gmail.com,FR,F,1172
5,22,email22@gmail.com,FR,F,1104
6,32,email32@gmail.com,FR,M,1094
7,54,email54@gmail.com,GB,F,1083
8,67,email67@gmail.com,DE,F,1079
9,39,email39@gmail.com,GB,M,1071


## Top 10 streamers as number of listens

In [6]:
metronomy_2016_FR_top_10_count = pd.read_sql_query("\
                        SELECT U.User_id, U.Email, U.Country, U.Gender, \
                        COUNT(Streams_duration) as Total_listen FROM STREAMS as ST \
                        INNER JOIN SONGS as SG \
                        ON SG.Sng_id = ST.Sng_id \
                        INNER JOIN ARTISTS as A \
                        ON A.Artist_id = SG.Artist_id \
                        INNER JOIN USERS as U \
                        ON U.User_id = ST.User_id \
                        WHERE A.Artist_name = 'Metronomy' \
                        AND ST.Country = 'FR' \
                        AND strftime('%Y', ST.Stream_date)= '2016' \
                        GROUP by U.User_id, U.Email, U.Country, U.Gender \
                        ORDER BY Total_listen DESC LIMIT 10", con)

In [7]:
metronomy_2016_FR_top_10_count

,User_id,Email,Country,Gender,Total_listen
0,77,email77@gmail.com,GB,F,10
1,15,email15@gmail.com,FR,M,7
2,32,email32@gmail.com,FR,M,7
3,71,email71@gmail.com,BR,F,7
4,92,email92@gmail.com,BR,M,7
5,22,email22@gmail.com,FR,F,6
6,29,email29@gmail.com,DE,M,6
7,30,email30@gmail.com,BR,F,6
8,54,email54@gmail.com,GB,F,6
9,58,email58@gmail.com,GB,F,6


# Number of days between the inscription and the 100th stream of all the Brazilian users registered in January 2018

In [8]:
BR_100 = pd.read_sql_query("""\
                        SELECT U.User_id,R.Stream_date,U.Inscription_date, \
                        julianDay(R.Stream_date) - julianDay(U.Inscription_date) as num_days FROM \
                        (SELECT User_id, Stream_date, \
                        RANK() OVER(PARTITION BY User_id ORDER BY Stream_date ASC) RANK FROM STREAMS) R \
                        INNER JOIN USERS U
                        ON R.User_ID = U.User_id
                        WHERE R.RANK = 100 \
                        AND U.country ='BR'
                        AND strftime('%Y-%m', U.Inscription_date) = '2018-01'
""",con)

In [9]:
BR_100

,User_id,Stream_date,Inscription_date,num_days
0,66,2019-07-10,2018-01-01,555.0
